<a href="https://colab.research.google.com/github/YunSeoHwan/My-own-study-repository/blob/main/%5BPytorch_Baseline%5D_EfficientNet_B0%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00


In [ ]:
# gpu, cpu 선택
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':0.01,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [ ]:
all_img_list = glob.glob('/content/drive/MyDrive/dacon_data/Vision/train/*/*')

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])

# img를 path로 저장
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

In [ ]:
df['img_path']

0       /content/drive/MyDrive/dacon_data/Vision/train...
1       /content/drive/MyDrive/dacon_data/Vision/train...
2       /content/drive/MyDrive/dacon_data/Vision/train...
3       /content/drive/MyDrive/dacon_data/Vision/train...
4       /content/drive/MyDrive/dacon_data/Vision/train...
                              ...                        
3452    /content/drive/MyDrive/dacon_data/Vision/train...
3453    /content/drive/MyDrive/dacon_data/Vision/train...
3454    /content/drive/MyDrive/dacon_data/Vision/train...
3455    /content/drive/MyDrive/dacon_data/Vision/train...
3456    /content/drive/MyDrive/dacon_data/Vision/train...
Name: img_path, Length: 3457, dtype: object

In [ ]:
# split data
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [ ]:
# label_encoding
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [ ]:
train['img_path']

945     /content/drive/MyDrive/dacon_data/Vision/train...
1237    /content/drive/MyDrive/dacon_data/Vision/train...
286     /content/drive/MyDrive/dacon_data/Vision/train...
2247    /content/drive/MyDrive/dacon_data/Vision/train...
1193    /content/drive/MyDrive/dacon_data/Vision/train...
                              ...                        
1781    /content/drive/MyDrive/dacon_data/Vision/train...
619     /content/drive/MyDrive/dacon_data/Vision/train...
2120    /content/drive/MyDrive/dacon_data/Vision/train...
2872    /content/drive/MyDrive/dacon_data/Vision/train...
586     /content/drive/MyDrive/dacon_data/Vision/train...
Name: img_path, Length: 2419, dtype: object

## CustomDataset

In [ ]:
class CustomDataset(Dataset):

    # transforms : 데이터 증강
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)
        

In [ ]:
# mix_up
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None, alpha=1.0, resize=(224, 224)):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.alpha = alpha
        self.resize = resize
        
    def mixup(self, image, label, resize):
        # randomly select another image and label
        index = np.random.randint(0, len(self.img_path_list))
        mix_image = cv2.imread(self.img_path_list[index])
        mix_label = self.label_list[index]

        # compute mixing coefficient
        lam = np.random.beta(self.alpha, self.alpha)

        # resize mix_image to the same size as image
        mix_image = cv2.resize(mix_image, resize)

        # transpose mix_image so that it has shape (3,224,224)
        mix_image = mix_image.transpose(2, 0, 1)

        # mix images and labels
        mix_image = lam * image + (1 - lam) * mix_image
        mix_label = lam * label + (1 - lam) * mix_label

        return mix_image, mix_label
    
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)
        image = cv2.resize(image, self.resize)  # resize image to fixed size

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            mix_image, mix_label = self.mixup(image, label, self.resize)
            return mix_image, mix_label
        else:
            return image

        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
# 데이터 증강 -> Albumentations 라이브러리 compose
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),      # 입력 이미지를 IMG_SIZE로 resize
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),   # 픽셀값 정규화
                            ToTensorV2()    # img to tensor
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
# num_workers : 기본 0 병렬처리 관련

train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform, alpha=0.5, resize=(224, 224))
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform, alpha=0.5, resize=(224, 224))
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [ ]:
# efficientnet_b0 모델 정의
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()

        # imageNet dataset 사용
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)  # fully connected layer
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x


In [ ]:
# efficientnet_b7 모델 정의
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()

        # ImageNet dataset 사용
        self.backbone = models.efficientnet_b7(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)  # fully connected layer
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x


## Train

In [ ]:

def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model


In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [ ]:
model = BaseModel()
model.eval()    # 평가모드
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

OutOfMemoryError: ignored

## Inference

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/dacon_data/Vision/test.csv')

In [ ]:
all_img_list = glob.glob('/content/drive/MyDrive/dacon_data/Vision/test/*')

In [ ]:
# img를 path로 저장
test['img_path'] = all_img_list

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/dacon_data/Vision/sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
submit.loc[submit['label'] == '0', 'label'] = '가구수정'
submit.loc[submit['label'] == '1', 'label'] = '걸레받이수정'
submit.loc[submit['label'] == '2', 'label'] = '곰팡이'
submit.loc[submit['label'] == '3', 'label'] = '꼬임'
submit.loc[submit['label'] == '4', 'label'] = '녹오염'
submit.loc[submit['label'] == '5', 'label'] = '들뜸'
submit.loc[submit['label'] == '6', 'label'] = '면불량'
submit.loc[submit['label'] == '7', 'label'] = '몰딩수정'
submit.loc[submit['label'] == '8', 'label'] = '반점'
submit.loc[submit['label'] == '9', 'label'] = '석고수정'
submit.loc[submit['label'] == '10', 'label'] = '오염'
submit.loc[submit['label'] == '11', 'label'] = '오타공'
submit.loc[submit['label'] == '12', 'label'] = '울음'
submit.loc[submit['label'] == '13', 'label'] = '이음부불량'
submit.loc[submit['label'] == '14', 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == '15', 'label'] = '터짐'
submit.loc[submit['label'] == '16', 'label'] = '틈새과다'
submit.loc[submit['label'] == '17', 'label'] = '피스'
submit.loc[submit['label'] == '18', 'label'] = '훼손'

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)